# Projet : Analysons des factures d'énergie 🌠🌠

Objectif : transformer un fichier de factures d'énergie en créeant des indicateurs permettant une prise de décision 

## 1 . Import et analyse du jeu de données

Source : https://drive.google.com/file/d/1Xpfmecb4PmW-CEqFuiLdelZx4jWxZTD-/view?usp=sharing


In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv('Invoice_20201220.csv', on_bad_lines='skip', delimiter="|")
#vérifications du type de données par colonne
print(df.dtypes)
df.head()

SITE_NAME       object
INVOICE_NAME    object
BEGIN           object
END             object
KWH_PERIOD      object
PERIOD_EUR      object
dtype: object


,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"


## 2. Transformation du jeu de données

**Colonne SITE_NAME : le format est composé de 5 chiffres et 2 lettres. Il faut supprimer tout autre caractère**

In [21]:
#analyse
print("Nombre de valeurs uniques avant modifications :", len(df['SITE_NAME'].unique()))
# affichage des valeurs nulles
print("\nAffichage des valeurs nulles:")
display(df.loc[df['SITE_NAME'].isnull()])

Nombre de valeurs uniques avant modifications : 2404

Affichage des valeurs nulles:


,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR


In [22]:
#transformation
df['SITE_NAME'] = df['SITE_NAME'].str.replace(r'^\d+_\w+ \((\w+)\)$', r'\1', regex=True)

print("Nombre de valeurs uniques après modifications :", len(df['SITE_NAME'].unique()))
display(df.head())

# The regular expression pattern r'^\d+_\w+ \((\w+)\)$' matches strings that start with one or more digits (\d+), followed by an underscore (_), one or more word characters (\w+), a space, 
# an open parenthesis (\(), one or more word characters (\w+), and a close parenthesis (\)), and captures the second set of word characters inside a group ((\w+)). 
# 
# The replacement pattern r'\1' replaces the entire match with the captured group (i.e., the second set of word characters).

Nombre de valeurs uniques après modifications : 2403


,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,1300082549,01/05/2018,31/05/2018,2 574,"333,37"


**Colonne INVOICE_NAME : supprimer la colonne**

In [23]:
df.pop('INVOICE_NAME')
df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,0001BB,01/01/2018,31/01/2018,2 447,"316,46"
1,0001BB,01/02/2018,28/02/2018,2 211,"286,82"
2,0001BB,01/03/2018,31/03/2018,2 450,"317,32"
3,0001BB,01/04/2018,30/04/2018,2 431,"315,00"
4,0001BB,01/05/2018,31/05/2018,2 574,"333,37"


**Ajouter une colonne qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.**

In [24]:
#analyse 
print("\nAffichage des valeurs nulles de la colonne BEGIN:")
display(df.loc[df['BEGIN'].isnull()])

print("\nAffichage des valeurs nulles de la colonne END:")
display(df.loc[df['END'].isnull()])


Affichage des valeurs nulles de la colonne BEGIN:


,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR



Affichage des valeurs nulles de la colonne END:


,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR


In [25]:
#convertir colonnes BEGIN et END en dates
df['BEGIN'] = pd.to_datetime(df['BEGIN'], dayfirst=True)
df['END'] = pd.to_datetime(df['END'], dayfirst=True)

#ajout de la colonne DAYS_PERIOD
df['DAYS_PERIOD'] = (df['END']-df['BEGIN']).dt.days+1
df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_PERIOD
0,0001BB,2018-01-01,2018-01-31,2 447,"316,46",31
1,0001BB,2018-02-01,2018-02-28,2 211,"286,82",28
2,0001BB,2018-03-01,2018-03-31,2 450,"317,32",31
3,0001BB,2018-04-01,2018-04-30,2 431,"315,00",30
4,0001BB,2018-05-01,2018-05-31,2 574,"333,37",31


**Ajouter une colonne qui donne la consommation journalière moyenne de la période**

In [26]:
# affichage des valeurs nulles
print("\nAffichage des valeurs nulles:")
display(df.loc[df['KWH_PERIOD'].isnull()])


Affichage des valeurs nulles:


,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_PERIOD


In [27]:
#supprimer les espaces de la colonne KWH_PERIOD
df['KWH_PERIOD'] = df['KWH_PERIOD'].str.replace(r'\s+', '', regex=True)
#The regular expression pattern r'\s+' matches one or more whitespace characters, including spaces, tabs, and newlines. 
#The regex=True argument tells str.replace() to interpret the pattern as a regular expression.

#convertir colonne KWH_PERIOD en numeric
df['KWH_PERIOD'] = pd.to_numeric(df['KWH_PERIOD'])


#ajout de la colonne KWH_DAILY
df['KWH_DAILY'] = round(df['KWH_PERIOD']/df['DAYS_PERIOD'], 1)
df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_PERIOD,KWH_DAILY
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.9
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,79.0
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.0
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.0
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.0


**Ajouter une colonne donnant pour chaque antenne, le nombre total de factures (même si l'info se répète à chaque ligne pour chaque antenne)**

In [28]:
#compter le nombre de factures par site
SeriesTotalInvoice = df['SITE_NAME'].value_counts()
#utiliser la fonction map permet d'appliquer la valeur de SeriesTotalInvoice à chaque fois que l'index de SeriesTotalInvoice correspond à la valeur de la colonne SITE_NAME
df['TOTAL_INVOICE'] = df['SITE_NAME'].map(SeriesTotalInvoice)
df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_PERIOD,KWH_DAILY,TOTAL_INVOICE
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.9,35
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,79.0,35
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.0,35
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.0,35
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.0,35


**Obtenir pour chaque antenne le coeff de variation de la consommation.**  
*Cela permettra d'avoir une idée de la dispersion de consommation d'énergie*  
  
Le coefficient de variation, également nommé écart type relatif, est défini comme le rapport entre l'écart-type et la moyenne  
CV = ecart-type / moyenne


In [34]:
meanBySite = round(df.groupby(['SITE_NAME'])['KWH_DAILY'].mean(),2)
stdBySite = round(df.groupby(['SITE_NAME'])['KWH_DAILY'].std(),2)
cvBySite = round(100*stdBySite / meanBySite, 2)
df['CV_SITE'] = df['SITE_NAME'].map(cvBySite)
df.head()

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS_PERIOD,KWH_DAILY,TOTAL_INVOICE,CV_SITE
0,0001BB,2018-01-01,2018-01-31,2447,"316,46",31,78.9,35,6.62
1,0001BB,2018-02-01,2018-02-28,2211,"286,82",28,79.0,35,6.62
2,0001BB,2018-03-01,2018-03-31,2450,"317,32",31,79.0,35,6.62
3,0001BB,2018-04-01,2018-04-30,2431,"315,00",30,81.0,35,6.62
4,0001BB,2018-05-01,2018-05-31,2574,"333,37",31,83.0,35,6.62


## 3. Exporter sous format CSV

In [33]:
#export du df sous format csv. Index = false évite de créer une colonne index pour les équipes compta qui y comprennent rien 
df.to_csv('invoices_v2.csv', index=False)